In [120]:

!sudo apt install sqlite3
!cp $SQLITE_STORE_DIR/faiss_store.db .
!ls -lh faiss_store.db

counter = !cat counter.txt
counter = int(counter[0]) - 1
%env MEMORY_BANK_NAME=bank_{counter}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
-rw-r--r-- 1 ubuntu ubuntu 179M Dec  7 23:40 faiss_store.db
env: MEMORY_BANK_NAME=bank_30


In [121]:
%%bash
echo "select value from kvstore where key = 'faiss_index:v1::${MEMORY_BANK_NAME}';" \
  | sqlite3 faiss_store.db \
  > faiss_index.json


In [122]:
!ls -lh faiss_index.json

-rw-rw-r-- 1 ubuntu ubuntu 14M Dec  7 23:40 faiss_index.json


In [123]:
import json
from IPython.display import display

# Load into dict
faiss_index = json.load(open('faiss_index.json'))
display(faiss_index.keys())
display(len(faiss_index['id_by_index']))
display(len(faiss_index['chunk_by_index']))
display(len(faiss_index['faiss_index']))

# Squad: 252 so mostly all contexts are a single chunk
# Quac: has 527 chunks so about 2 chunks per context
# NarrativeQA: 274 chunks so about 2.4 chunks per context

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

274

274

14030300

In [61]:
import os
from llama_stack_client import LlamaStackClient

# Test run
memory_bank_name = os.getenv("MEMORY_BANK_NAME")

client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=memory_bank_name,
    query=["Hello World"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content='The Bey Hive is the name given to Beyoncé\'s fan base. Fans were previously titled "The Beyontourage", (a portmanteau of Beyoncé and entourage). The name Bey Hive derives from the word beehive, purposely misspelled to resemble her first name, and was penned by fans after petitions on the online social networking service Twitter and online news reports during competitions.', document_id='8', token_count=80), Chunk(content="Established in 1988, Chengdu Hi-tech Industrial Development Zone (Chinese: 成都高新技术产业开发区; pinyin: Chéngdū Gāoxīn Jìshù Chǎnyè Kāifā Qū) was approved as one of the first national hi-tech development zones in 1991. In 2000, it was open to APEC and has been recognized as a national advanced hi-tech development zone in successive assessment activities held by China's Ministry of Science and Technology. It ranks 5th among the 53 national hi-tech development zones in China in terms of comprehensive strength.", document_id='184', t

Chunk(content='The Bey Hive is the name given to Beyoncé\'s fan base. Fans were previously titled "The Beyontourage", (a portmanteau of Beyoncé and entourage). The name Bey Hive derives from the word beehive, purposely misspelled to resemble her first name, and was penned by fans after petitions on the online social networking service Twitter and online news reports during competitions.', document_id='8', token_count=80)
0.5918109683380094
Chunk(content="Established in 1988, Chengdu Hi-tech Industrial Development Zone (Chinese: 成都高新技术产业开发区; pinyin: Chéngdū Gāoxīn Jìshù Chǎnyè Kāifā Qū) was approved as one of the first national hi-tech development zones in 1991. In 2000, it was open to APEC and has been recognized as a national advanced hi-tech development zone in successive assessment activities held by China's Ministry of Science and Technology. It ranks 5th among the 53 national hi-tech development zones in China in terms of comprehensive strength.", document_id='184', token_count=13

In [111]:
import pandas as pd

df = pd.read_parquet("data/squad_250.parquet")
display(df.iloc[87]["questions"])
display(df.iloc[87]["context"])
df.head()

array(['When was the first transistor created?',
       'Who invented the first transistor?',
       'When was the first transistor implemented for practical use?',
       'When did the implementers receive a Nobel Prize for making the transistor?',
       'What list is the transistor on?'], dtype=object)

'The transistor is the fundamental building block of modern electronic devices, and is ubiquitous in modern electronic systems. First conceived by Julius Lilienfeld in 1926 and practically implemented in 1947 by American physicists John Bardeen, Walter Brattain, and William Shockley, the transistor revolutionized the field of electronics, and paved the way for smaller and cheaper radios, calculators, and computers, among other things. The transistor is on the list of IEEE milestones in electronics, and Bardeen, Brattain, and Shockley shared the 1956 Nobel Prize in Physics for their achievement.'

,context,questions,answers
0,"Sometimes, poly-sided matches that pit every o...","[What happens in a fatal four-way?, In a fatal...","[four wrestlers, each for themselves, fight in..."
1,"In Lebanon, a part of the Christian population...",[Lebanese people of what religion sometimes co...,"[Christian, Arabic, Lebanese Arabic, Arabic, L..."
2,"The current ""Precentor"" (Head of Music) is Tim...","[What term is given to the Head of Music?, Wha...","[""Precentor"", didgeridoo, Tim Johnson, eight]"
3,Israeli universities are among 100 top world u...,[Israeli universities rank where in mathematic...,"[100, six, stem-cell research]"
4,"The current Chief of the Defence Staff, the pr...",[What is the name of the person who is the cur...,"[General Sir Nicholas Houghton, Chief of the D..."


In [73]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:5]

1174
1174


['What happens in a fatal four-way?',
 'In a fatal four-way, who can make tags?',
 'What was the most common type of match before the fatal four-way? ',
 'Why are tagging rules made?',
 'Lebanese people of what religion sometimes consider Lebanese to be a distinct language?']

[0, 0, 0, 0, 1]

In [102]:
from tqdm import tqdm

top_k = 5

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=memory_bank_name,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

100%|████████████████████████████████████████████████████████████████████████████| 1174/1174 [01:19<00:00, 14.68it/s]


[[0, 89, 177, 216, 222],
 [0, 42, 248, 177, 190],
 [0, 143, 213, 120, 9],
 [0, 42, 190, 58, 239],
 [1, 247, 231, 5, 80],
 [1, 42, 80, 248, 102],
 [1, 231, 80, 247, 86],
 [1, 231, 247, 5, 80],
 [1, 248, 80, 102, 53],
 [2, 152, 227, 230, 114],
 [2, 114, 227, 230, 102],
 [2, 13, 87, 225, 4],
 [2, 84, 235, 167, 133],
 [3, 208, 212, 29, 144],
 [3, 87, 67, 22, 142],
 [3, 22, 47, 232, 201],
 [4, 50, 194, 229, 76],
 [4, 50, 76, 194, 221],
 [4, 50, 194, 245, 76],
 [4, 194, 50, 229, 76],
 [5, 230, 147, 247, 210],
 [5, 230, 147, 128, 59],
 [5, 210, 247, 230, 231],
 [5, 150, 230, 247, 105],
 [5, 150, 105, 31, 210],
 [6, 126, 190, 20, 50],
 [6, 244, 20, 168, 191],
 [6, 168, 41, 29, 201],
 [6, 168, 194, 20, 191],
 [6, 29, 187, 20, 244],
 [6, 245, 194, 180, 20],
 [6, 29, 201, 41, 168],
 [6, 29, 187, 20, 244],
 [6, 190, 126, 244, 150],
 [6, 29, 201, 41, 168],
 [7, 195, 174, 162, 186],
 [7, 222, 108, 16, 24],
 [7, 222, 75, 16, 43],
 [7, 108, 164, 17, 220],
 [7, 101, 164, 108, 220],
 [8, 159, 152, 137, 

In [103]:
import numpy as np

res = np.array(results)
res.shape

(1174, 5)

In [104]:
targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(1174,)

(1174, 1)

(1174, 5)

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [106]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.1998296422487223)

In [107]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.9923339011925043)

In [108]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.9737365133446904)